安装MindSpore框架和MindNLP套件

In [ ]:
!pip install pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/unified/aarch64/mindspore-2.5.0-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install mindnlp==0.4.0

In [ ]:
!pip install jieba

In [ ]:
%env HF_ENDPOINT=https://hf-mirror.com

In [ ]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn
mindspore.set_context(device_target='Ascend', device_id=0)

from mindnlp.dataset import load_dataset

from mindnlp.engine.trainer import Trainer

In [ ]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']
# 为加快运行速度只选取一部分训练
imdb_train, _ = imdb_train.split([0.1, 0.9])
imdb_test, _ = imdb_test.split([0.1, 0.9])

In [ ]:
imdb_train.get_dataset_size()

In [ ]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

In [ ]:
from mindnlp.transformers import OpenAIGPTTokenizer
# tokenizer
gpt_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [ ]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [ ]:
next(dataset_train.create_tuple_iterator())

In [ ]:
import evaluate
import numpy as np
from mindnlp.engine.utils import EvalPrediction

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from mindnlp.transformers import OpenAIGPTForSequenceClassification
from mindnlp.engine import TrainingArguments

model = OpenAIGPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

training_args = TrainingArguments(
    "./output/gpt",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=200,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(model=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, compute_metrics=compute_metrics,
                  args=training_args)

#### 注: 如果想要运行的更快一些，可以在训练时需要V100的算力

In [ ]:
# trainer.run(tgt_columns="labels")
trainer.train()

In [ ]:
# evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
# evaluator.run(tgt_columns="labels")
from tqdm import tqdm

def compute_accuracy(logits, labels):
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def evaluate_fn(model, test_dataset):
    total = test_dataset.get_dataset_size()
    epoch_acc = 0
    step_total = 0
    model.set_train(False)

    with tqdm(total=total) as progress_bar:
        for batch in test_dataset.create_dict_iterator():
            label = batch.pop('labels')
            logits = model(**batch).logits

            acc = compute_accuracy(logits, label)['accuracy']
            epoch_acc += acc

            step_total += 1
            acc=epoch_acc/step_total

    return acc

In [ ]:
acc = evaluate_fn(model, dataset_val)
print(f"Accuracy: {acc}")